In [112]:
import pandas as pd
import json
import operator
import datetime as dt

In [113]:
movie_df = pd.read_csv('tmdb_5000_movies.csv')
credit_df = pd.read_csv('tmdb_5000_credits.csv')
df = pd.concat([movie_df, credit_df], axis=1)

In [114]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'title', 'cast', 'crew'],
      dtype='object')

In [164]:
df = df[df['revenue']>0]

In [166]:
df['budget']=df['budget'].replace(0,df['budget'].mean())

/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [167]:
len(df)

3376

In [168]:
def is_json(myjson):
    try:
        json_object = json.loads(myjson)
    except:
        return False
    return True

def encode_json_column(pandas_data_frame, json_column_index=0, json_id_column="id", encodinglimit = 1000, remove_non_encoded = 1):
      
    X = pandas_data_frame.iloc[:, :].values
    all_encodedcolumns = {}
    
    for row in X:                    
        if(is_json(row[json_column_index])): #some data is just not json. ignore            
            #for each feature in the json
            for json_features in json.loads(row[json_column_index]):
                #pick out its id (the json identifier you specifc in json_id_column)
                featureid = json_features[json_id_column]                
                #if this id hasn't been seen yet, add it to the dataframe with default 0
                if featureid not in all_encodedcolumns:
                    all_encodedcolumns[featureid] = 1                   
                #else just set it to 1 here
                all_encodedcolumns[featureid] += 1

    top_encodedcolumns = sorted(all_encodedcolumns.items(), key=operator.itemgetter(1), reverse=True)
    
    if encodinglimit < len(top_encodedcolumns):
        top_encodedcolumns = top_encodedcolumns[:encodinglimit]        

    top_encodedcolumns = dict(top_encodedcolumns)

    #keep track of whether a column has been encoded into the dataframe already, else we'd reset all the values to 0
    df_encodedcolumns = []
    count = 0
    
    #for each row in the data
    for row in X:       
        #keep track of whether this row can be kept or not, based on if it has an encoded value
        has_an_encoded_value = 0        
        if(is_json(row[json_column_index])): #some data is just not json. ignore         
            #for each feature in the json
            for json_features in json.loads(row[json_column_index]):  
                #pick out its id (the json identifier you specifc in json_id_column)
                featureid = json_features[json_id_column]                           
                if featureid in top_encodedcolumns:
                    #if this id hasn't been seen yet, add it to the dataframe with default 0
                    if featureid not in df_encodedcolumns:
                        df_encodedcolumns.append(featureid)
                        pandas_data_frame[featureid]=0                  
                    #else just set it to 1 here
                    pandas_data_frame[featureid][count] = 1                    
                    has_an_encoded_value = 1    
        if has_an_encoded_value == 0 & remove_non_encoded == 1:
            pandas_data_frame.drop(pandas_data_frame.index[count])
        else:
            count+=1
            
    return pandas_data_frame

In [169]:
## transform genre
encode_json_column(df, 1, "name", 30, 1)

/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,Romania,Mexico,Switzerland,Denmark,South Korea,Brazil,Luxembourg,release_date_datetime,release_year,release_month
0,237000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,0,0,0,0,0,0,0,2009-12-10,2009,12
1,300000000.00,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,0,0,0,0,0,0,0,2007-05-19,2007,5
2,245000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.38,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,0,0,0,0,0,0,0,2015-10-26,2015,10
3,250000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,0,0,0,0,0,0,0,2012-07-16,2012,7
4,260000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.93,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,0,0,0,0,0,0,0,2012-03-07,2012,3
5,258000000.00,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.70,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,0,0,0,0,0,0,0,2007-05-01,2007,5
6,260000000.00,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.68,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,0,0,0,0,0,0,0,2010-11-24,2010,11
7,280000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/193/avengers_ag...,99861,"[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...",en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.28,"[{""name"": ""Marvel Studios"", ""id"": 420}, {""name...",...,0,0,0,0,0,0,0,2015-04-22,2015,4
8,250000000.00,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://harrypotter.warnerbros.com/harrypottera...,767,"[{""id"": 616, ""name"": ""witch""}, {""id"": 2343, ""n...",en,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...",98.89,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...",...,0,0,0,0,0,0,0,2009-07-07,2009,7
9,250000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.batmanvsupermandawnofjustice.com/,209112,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 7002...",en,Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,155.79,"[{""name"": ""DC Comics"", ""id"": 429}, {""name"": ""A...",...,0,0,0,0,0,0,0,2016-03-23,2016,3


In [170]:
## transform country
encode_json_column(df, 10, "name", 30, 1)

/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,Romania,Mexico,Switzerland,Denmark,South Korea,Brazil,Luxembourg,release_date_datetime,release_year,release_month
0,237000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,0,0,0,0,0,0,0,2009-12-10,2009,12
1,300000000.00,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,0,0,0,0,0,0,0,2007-05-19,2007,5
2,245000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.38,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,0,0,0,0,0,0,0,2015-10-26,2015,10
3,250000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,0,0,0,0,0,0,0,2012-07-16,2012,7
4,260000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.93,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,0,0,0,0,0,0,0,2012-03-07,2012,3
5,258000000.00,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.70,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,0,0,0,0,0,0,0,2007-05-01,2007,5
6,260000000.00,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.68,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,0,0,0,0,0,0,0,2010-11-24,2010,11
7,280000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/193/avengers_ag...,99861,"[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...",en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.28,"[{""name"": ""Marvel Studios"", ""id"": 420}, {""name...",...,0,0,0,0,0,0,0,2015-04-22,2015,4
8,250000000.00,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://harrypotter.warnerbros.com/harrypottera...,767,"[{""id"": 616, ""name"": ""witch""}, {""id"": 2343, ""n...",en,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...",98.89,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...",...,0,0,0,0,0,0,0,2009-07-07,2009,7
9,250000000.00,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.batmanvsupermandawnofjustice.com/,209112,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 7002...",en,Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,155.79,"[{""name"": ""DC Comics"", ""id"": 429}, {""name"": ""A...",...,0,0,0,0,0,0,0,2016-03-23,2016,3


In [171]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'title', 'cast', 'crew', 'Action',
       'Adventure', 'Fantasy', 'Science Fiction', 'Crime', 'Drama', 'Thriller',
       'Animation', 'Family', 'Western', 'Comedy', 'Romance', 'Horror',
       'Mystery', 'History', 'War', 'Music', 'Documentary', 'Foreign',
       'TV Movie', 'United States of America', 'United Kingdom',
       'Czech Republic', 'New Zealand', 'Germany', 'China', 'Canada', 'Italy',
       'Japan', 'Australia', 'France', 'Belgium', 'India', 'Netherlands',
       'Spain', 'United Arab Emirates', 'Hong Kong', 'Ireland', 'Hungary',
       'Norway', 'Sweden', 'South Africa', 'Russia', 'Romania', 'Mexico',
       'Switzerland', 'Denmark', 'South Korea',

In [172]:
df['release_date_datetime'] = pd.to_datetime(df['release_date'])

/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [173]:
df['release_year'], df['release_month'] = df['release_date_datetime'].dt.year, df['release_date_datetime'].dt.month

/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [175]:
df['release_year'], df['release_month'] = df['release_year'].fillna(0.0).astype(int), df['release_month'].fillna(0.0).astype(int)

/Users/ShiMengjie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [177]:
df.to_csv('dataset_combined_w_dummies.csv')

In [178]:
inflation = pd.read_excel('inflation.xlsx')

In [179]:
inflation.head()

,YEAR,AVE Inflation,Relative-Year2020 Value
0,1914,1.00,0.04
1,1915,1.00,0.04
2,1916,7.90,0.04
3,1917,17.40,0.05
4,1918,18.00,0.06


In [180]:
df_with_inf = df.merge(inflation, left_on='release_year', right_on='YEAR')

In [183]:
df_with_inf['budget_relative_2020'] = df_with_inf['budget']*df_with_inf['Relative-Year2020 Value']

In [184]:
df_with_inf['revenue_relative_2020'] = df_with_inf['revenue']*df_with_inf['Relative-Year2020 Value']

In [185]:
df_with_inf.to_csv('combined_inflation.csv')